# LCEL 체인에서 스트리밍 사용하기

이 노트북에서는 **LCEL 선언적 체인**에서 **스트리밍**을 사용하는 방법을 알아봅니다.

## LCEL 스트리밍의 장점

LCEL로 구성한 체인은 **자동으로 스트리밍을 지원**합니다.

```python
chain = template | model

# 별도 설정 없이 바로 스트리밍 가능!
for chunk in chain.stream(inputs):
    print(chunk)
```

## @chain + yield vs LCEL stream() 비교

| 방식 | 코드 | 특징 |
|------|------|------|
| **@chain + yield** | 함수 내부에 `yield` 작성 | 명시적, 커스텀 가능 |
| **LCEL stream()** | `chain.stream()` 호출 | 자동, 간편함 |

```python
# @chain + yield (12번 노트북)
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token

# LCEL stream() (이 노트북)
chain = template | model
for token in chain.stream(inputs):  # 자동 지원!
    print(token)
```

## 스트리밍 지원 컴포넌트

모든 LangChain 컴포넌트가 스트리밍을 지원하는 것은 아닙니다:

| 컴포넌트 | 스트리밍 지원 | 동작 |
|----------|--------------|------|
| ChatModel | O | 토큰 단위 스트리밍 |
| LLM | O | 토큰 단위 스트리밍 |
| PromptTemplate | X | 최종 결과만 전달 |
| OutputParser | 일부 | 파서에 따라 다름 |

---

# 1. Ollama 설치 및 서버 실행

In [ ]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [ ]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 구성 요소 준비 및 체인 구성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Prompt Template
template = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 어시스턴트입니다.'),
    ('human', '{question}'),
])

# Model
model = ChatOllama(model='llama3.2')

# LCEL로 체인 구성
chatbot = template | model

# 4. LCEL 체인 스트리밍 실행

**핵심 코드:**

```python
for part in chatbot.stream({'question': '...'}):
    print(part)
```

- LCEL 체인은 **자동으로** 스트리밍 지원
- 별도의 `yield` 코드 작성 불필요
- 각 `part`는 **AIMessageChunk** 객체

In [ ]:
# LCEL 체인 스트리밍
print("=== LCEL 스트리밍 (전체 객체) ===")

for part in chatbot.stream({'question': '거대 언어 모델은 어디서 제공하나요?'}):
    print(part)

# 5. 실시간 텍스트 출력

ChatGPT처럼 글자가 하나씩 나타나는 효과를 구현합니다.

In [ ]:
# 실시간 텍스트 출력
print("=== 실시간 텍스트 출력 ===")

for part in chatbot.stream({'question': 'Python의 장점 3가지를 알려주세요.'}):
    print(part.content, end='', flush=True)

print()  # 줄바꿈

# 6. 각 chunk 상세 확인

스트리밍에서 각 `part`의 구조를 확인합니다.

In [ ]:
# 각 chunk 상세 분석
print("=== 각 chunk 상세 ===")

chunks = []
for i, part in enumerate(chatbot.stream({'question': '안녕!'})):
    chunks.append(part)
    if i < 5:  # 처음 5개만 출력
        print(f"[{i}] type: {type(part).__name__}")
        print(f"    content: '{part.content}'")
        print()

print(f"... 총 {len(chunks)}개 chunks")
print(f"\n전체 응답: {''.join([c.content for c in chunks])}")

# 7. StrOutputParser와 함께 스트리밍

`StrOutputParser`를 추가하면 AIMessageChunk 대신 **문자열**로 스트리밍됩니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# OutputParser 추가 체인
chain_with_parser = template | model | StrOutputParser()

print("=== StrOutputParser 스트리밍 ===")

for part in chain_with_parser.stream({'question': 'Hello!'}):
    print(f"type: {type(part).__name__}, value: '{part}'")

In [ ]:
# StrOutputParser로 깔끔한 실시간 출력
print("=== StrOutputParser 실시간 출력 ===")

for text in chain_with_parser.stream({'question': 'LangChain이 뭔가요?'}):
    print(text, end='', flush=True)

print()

# 8. 비동기 스트리밍 (astream)

비동기 환경에서는 `astream()`을 사용합니다.

In [ ]:
# 비동기 스트리밍
print("=== 비동기 스트리밍 (astream) ===")

async for part in chatbot.astream({'question': '짧게 인사해주세요.'}):
    print(part.content, end='', flush=True)

print()

---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI()

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 스트리밍 방식 비교 정리

| 노트북 | 방식 | 코드 | 특징 |
|--------|------|------|------|
| 10 | 모델 직접 | `model.stream()` | 가장 기본 |
| 12 | @chain + yield | `for token in model.stream(): yield token` | 커스텀 가능 |
| **15** | **LCEL stream** | **`chain.stream()`** | **자동, 간편** |

## LCEL 스트리밍 추천 패턴

### 기본 패턴
```python
chain = prompt | model
for chunk in chain.stream(inputs):
    print(chunk.content, end='')
```

### 문자열로 변환
```python
chain = prompt | model | StrOutputParser()
for text in chain.stream(inputs):
    print(text, end='')
```

### 비동기 스트리밍
```python
async for chunk in chain.astream(inputs):
    print(chunk.content, end='')
```

## 언제 어떤 방식을 사용할까?

| 상황 | 추천 방식 |
|------|----------|
| 단순 스트리밍 | **LCEL `chain.stream()`** |
| 스트리밍 중 토큰 변환 | @chain + yield |
| 스트리밍 중 로깅 | @chain + yield |
| 웹 서버 (FastAPI) | LCEL `chain.astream()` |